In [ ]:
!pip install tensorflow-recommenders

In [1]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf
import pandas as pd

import tensorflow_recommenders as tfrs

In [2]:
# Load the user_rating.csv file
data = pd.read_csv('user_rating.csv')

In [5]:
# Create a tf.data.Dataset from the loaded data
ratings = tf.data.Dataset.from_tensor_slices({
    "plan_trip_id": data["plan_trip_id"].astype(str),
    "user_id": data["user_id"].astype(str),
    "rating": data["rating"].astype(str)
})
users = tf.data.Dataset.from_tensor_slices(data["user_id"])

In [7]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000,seed=42,reshuffle_each_iteration=False)
train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [10]:
plan_trip_id = ratings.batch(1_000_000).map(lambda x: x["plan_trip_id"])
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_plan_trip_id = np.unique(np.concatenate(list(plan_trip_id)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [11]:
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32

    # Compute embeddings for users.
    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # Compute embeddings for movies.
    self.plan_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_plan_trip_id, mask_token=None),
      tf.keras.layers.Embedding(len(unique_plan_trip_id) + 1, embedding_dimension)
    ])

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])

  def call(self, inputs):

    user_id, plan_trip_id = inputs

    user_embedding = self.user_embeddings(user_id)
    plan_embedding = self.plan_embeddings(plan_trip_id)

    return self.ratings(tf.concat([user_embedding, plan_embedding], axis=1))

In [12]:
task = tfrs.tasks.Ranking(
  loss = tf.keras.losses.MeanSquaredError(),
  metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

In [9]:
# Select the basic features
ratings = ratings.map(lambda x: {
    "plan_trip_id": x["plan_trip_id"],
    "user_id": x["user_id"]
})

users = users.map(lambda x: {
    "plan_trip_id": x
})

In [13]:
class PlanTripMod(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel()
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model(
        (features["user_id"], features["plan_trip_id"]))

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    labels = features.pop("rating")

    rating_predictions = self(features)

    # The task computes the loss and the metrics.
    return self.task(labels=labels, predictions=rating_predictions)

In [14]:
model = PlanTripMod()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [15]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [16]:
model.fit(cached_train, epochs=3)

Epoch 1/3


UnimplementedError: Graph execution error:

Detected at node 'ranking_1/mean_squared_error/Cast' defined at (most recent call last):
    File "d:\python\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "d:\python\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "d:\python\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "d:\python\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "d:\python\lib\site-packages\ipykernel\kernelapp.py", line 725, in start
      self.io_loop.start()
    File "d:\python\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "d:\python\lib\asyncio\base_events.py", line 603, in run_forever
      self._run_once()
    File "d:\python\lib\asyncio\base_events.py", line 1906, in _run_once
      handle._run()
    

In [ ]:
# Adapt the user_ids_vocabulary
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

# Adapt the plan_trip_id_vocabulary
plan_trip_id_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
plan_trip_id_vocabulary.adapt(ratings.map(lambda x: x["plan_trip_id"]))


In [ ]:
class PlanTripModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_model: tf.keras.Model,
      plan_trip_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.plan_trip_model = plan_trip_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_embeddings = self.user_model(features["user_id"])
    plan_trip_embeddings = self.plan_trip_model(features["plan_trip_id"])

    return self.task(user_embeddings, plan_trip_embeddings)

In [ ]:
# Define user and plan trip models.
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64,),
  
    tf.keras.layers.Dense(128, activation='relu')
 
])
plan_trip_model = tf.keras.Sequential([
    plan_trip_id_vocabulary,
    tf.keras.layers.Embedding(plan_trip_id_vocabulary.vocab_size(), 64),
    
    tf.keras.layers.Dense(128, activation='relu')
])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    candidates=ratings.batch(64).map(lambda x: x["plan_trip_id"]).map(plan_trip_model)
  )
)

In [ ]:
# Create a retrieval model.
model = PlanTripModel(user_model, plan_trip_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
model.fit(ratings.batch(4096), epochs=10)

# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    ratings.batch(100).map(lambda x: x["plan_trip_id"]).map(plan_trip_model))



Percobaan beda


In [ ]:
# Convert the model to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model to a file
with open('plan_trip_model.tflite', 'wb') as f:
    f.write(tflite_model)